### exploratory notebook

In [1]:
import pylast
import pickle
import time
import pandas as pd

In [4]:
with open('user_list.p', 'rb') as f:
    user_list = pickle.load(f)
    
with open('prelim_user_data.p', 'rb') as f:
    user_data = pickle.load(f)


## spotipy

### combining lastfm, spotify data

In [27]:
mask = df.user == 'wildaman1'
top10artists = df[mask].artist[:10].values

artist_features_dict = {artist: get_audio_feats_by_artist(artist) for artist in top10artists}
    
    

### get top songs

In [37]:
# goal: implement levenshtein distance to validate correct result
# go from lastfm tracks to spotify features for that track
import os
import sys
import json
import spotipy
import webbrowser
import time
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

from json.decoder import JSONDecodeError
import Levenshtein.StringMatcher as leven
import numpy as np

# set up spotipy api
client_id='75ceaa8e4ccd4810b1995ee82aafd5e3'
client_secret='04a0d2e14c424f259132ff86275e5503'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace=False

def get_features(artist_track, tolerance=10, verbose=False):
    """
    artist_track = 'artist name' + 'track name'
    
    returns: dictionary containing echonest features for that tracks
    """
    
    # make query
    results = sp.search(q=artist_track, limit=5)
    
    tracks = results['tracks']['items'] 
    
    # makes sure a match was returned:
    if tracks == []:
        print('no track found for {}'.format(artist_track))
        return 
    
    # make sure we have a convincing match
    best_distance = np.inf
    best_track = None
    for track in tracks:
        val_artist = track['artists'][0]['name']
        val_name = track['name']
        val_string = val_artist + ' ' + val_name
        val_string = val_string.lower()

        true_string = artist_track.lower()

        dist = leven.distance(true_string, val_string)
        
        
        if dist < best_distance:
            
            best_distance = dist
            best_track = track
            best_track_artist_track = val_string
            spotify_string = val_artist + ' ' + val_name
        
        
    if best_distance < tolerance:
        if verbose:
            print('===================')
            print('found match!')
            print('best distance:\t', best_distance)
            print('input:\t', artist_track)
            print('found:\t', spotify_string)
            print('====================')
            print()
        uri = best_track['uri']
        features = sp.audio_features([uri])
        return best_track_artist_track, features
        
    else:
        if verbose:
            print('+++++++++++++++++++')
            print(f'no match for {artist_track}')
            print()
            print('input:\t', artist_track)
            print('found:\t', spotify_string)
            print('+++++++++++++++++++')
        return None
    
test = "Caribou Can't Do Without"
test_dict = get_features(test)


### psycopg2 connection

In [13]:
import psycopg2


connection = psycopg2.connect(user="max",
                              password="password",
                              host="127.0.0.1",
                              port="5432",
                              database="final")
cursor = connection.cursor()

In [56]:
q = """
INSERT INTO test (customer, count) VALUES (%s,%s)
"""
record_to_insert = ('bill', 15)
cursor.execute(q, record_to_insert)

connection.commit()
count = cursor.rowcount
print (count, "Record inserted successfully into mobile table")

1 Record inserted successfully into mobile table


In [49]:
# schema for actual insertion
# ------------------------------------------
# lastfm_user | arist_track | percent_listen 
# ------------------------------------------

q = """
SELECT DISTINCT arist_track FROM lastfm LIMIT 50000;
"""

cursor.execute(q)

cursor.fetchall()


In [16]:
# schema for spotify data
# ---------------------------------------------------------
# lastfm_artist_track | spotify_artist_track | **features
# ---------------------------------------------------------
# =========================================================

# EXAMPLE:

# [{'danceability': 0.691,
#    'energy': 0.407,
#    'key': 0,
#    'loudness': -9.053,
#    'mode': 1,
#    'speechiness': 0.0473,
#    'acousticness': 0.792,
#    'instrumentalness': 0.642,
#    'liveness': 0.783,
#    'valence': 0.491,
#    'tempo': 128.039,
#    'type': 'audio_features',
#    'id': '3n4M80UmJNHws4ZqdM7KEF',
#    'uri': 'spotify:track:3n4M80UmJNHws4ZqdM7KEF',
#    'track_href': 'https://api.spotify.com/v1/tracks/3n4M80UmJNHws4ZqdM7KEF',
#    'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3n4M80UmJNHws4ZqdM7KEF',
#    'duration_ms': 236400,
#    'time_signature': 4}])

q = "DROP TABLE spotify_features"

cursor.execute(q)


q = """
CREATE TABLE spotify_features (
    lastfm_artist_track VARCHAR (255) NOT NULL,
    spotify_artist_track VARCHAR (255) NOT NULL,
    danceability NUMERIC NOT NULL,
    energy NUMERIC NOT NULL,
    key NUMERIC NOT NULL,
    loudness NUMERIC NOT NULL,
    mode NUMERIC NOT NULL,
    speechiness NUMERIC NOT NULL,
    acousticness NUMERIC NOT NULL,
    instrumentalness NUMERIC NOT NULL,
    liveness NUMERIC NOT NULL,
    valence NUMERIC NOT NULL,
    tempo NUMERIC NOT NULL,
    uri VARCHAR (255) NOT NULL,
    analysis_url VARCHAR (255) NOT NULL,
    duration_ms NUMERIC NOT NULL,
    time_signature SMALLINT NOT NULL
);
"""

cursor.execute(q)
connection.commit()

In [38]:
def make_spotify_row(lastfm_artist_track):
    test_dict = get_features(lastfm_artist_track)
    if test_dict == None:
        return None
    keys = [
        'danceability',
        'energy',
        'key',
        'loudness',
        'mode',
        'speechiness',
        'acousticness',
        'instrumentalness',
        'liveness',
        'valence',
        'tempo',
        'uri',
        'analysis_url',
        'duration_ms',
        'time_signature'
    ]

    spotify_artist_track = test_dict[0]
    d = test_dict[1][0]
    features = tuple(d[key] for key in keys)
    row = (lastfm_artist_track, spotify_artist_track) + features
    return row

make_spotify_row("Caribou Can't Do Without")

("Caribou Can't Do Without",
 "caribou can't do without you",
 0.691,
 0.407,
 0,
 -9.053,
 1,
 0.0473,
 0.792,
 0.642,
 0.783,
 0.491,
 128.039,
 'spotify:track:3n4M80UmJNHws4ZqdM7KEF',
 'https://api.spotify.com/v1/audio-analysis/3n4M80UmJNHws4ZqdM7KEF',
 236400,
 4)

In [39]:
def spotify_insert_row(lastfm_track):
    q = """
    INSERT INTO 
        spotify_features (
            lastfm_artist_track,
            spotify_artist_track,
            danceability,
            energy,
            key,
            loudness,
            mode,
            speechiness,
            acousticness,
            instrumentalness,
            liveness,
            valence,
            tempo,
            uri,
            analysis_url,
            duration_ms,
            time_signature
            ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """
    row = make_spotify_row(lastfm_track)
    if row == None:
        return None
        
    cursor.execute(q, row)
    connection.commit()
    
spotify_insert_row('bonobo the keeper')

In [34]:
q = """
SELECT DISTINCT arist_track FROM lastfm
LIMIT 5;
"""

cursor.execute(q)
lastfm_artist_tracks = cursor.fetchall()
lastfm_artist_tracks

[('𝖍𝖛𝒙𝖛𝖓 Celeste - XXX',),
 ('💊💊 FUCKBOY ANTHEM',),
 ("💊💊 HAD MY GUARD UP 2 U (SINCE '92)",),
 ('💊💊 MASS APPEAL',),
 ('💊💊 PARO',)]

In [41]:
# extract string from tuple
lastfm_artist_tracks = [t[0] for t in lastfm_artist_track]
for t in lastfm_artist_tracks:
    spotify_insert_row(t)

no track found for  
no track found for '
no track found for  
no track found for  


In [50]:
sample_features = []
for track in sample_tracks:
    track = track[0]
    feature = get_features(track, tolerance=5)
    time.sleep(.1)
    if feature:  # if not None
        sample_features.append(feature)
    
# plan for tomorrow:
# new table, track_features

# lastfm_name, spotify_name, *features

# the goal here would to be able to join on lastfm_name

# ---------------------------------------------------------

# also, make backups of code and data tomorrow if possible.

found match!
best distance:	 0
input:	 Abandon All Ships Shake Your AAS
found:	 Abandon All Ships Shake Your AAS

found match!
best distance:	 0
input:	 Abandon All Ships Strange Love
found:	 Abandon All Ships Strange Love

found match!
best distance:	 0
input:	 Abandon All Ships Structures
found:	 Abandon All Ships Structures

found match!
best distance:	 0
input:	 Abandon All Ships Take One Last Breath
found:	 Abandon All Ships Take One Last Breath

found match!
best distance:	 0
input:	 Abandon All Ships Trapped
found:	 Abandon All Ships Trapped

no track found for Abandon All Ships We'll Be Fine (Drake Cover)
found match!
best distance:	 0
input:	 Abandon All Ships When Dreams Become Nightmares
found:	 Abandon All Ships When Dreams Become Nightmares

found match!
best distance:	 1
input:	 Abandon All Ships brendons song
found:	 Abandon All Ships Brendon's Song

no track found for Abandon All Ships maria (i like it loud) (scooter cover)
found match!
best distance:	 1
input:	 Abandon

KeyboardInterrupt: 

In [54]:
connection.commit()